In [1]:
import os
import json
from tqdm import tqdm

In [39]:
tags_cate1_cate2 = json.load(open("D:/_WangKe/scikkk.github.io/projects/ganvana/tags_cate1_cate2.json", "r", encoding="utf-8"))
tags_cate1_cate2["tags"] = {item['id']: item for item in tags_cate1_cate2["tags"]}
id_cates = {item['id']: item for item in tags_cate1_cate2["cate1List"]+tags_cate1_cate2["cate2List"]}
name_cates = {item['name']: item for item in tags_cate1_cate2["cate1List"]+tags_cate1_cate2["cate2List"]}

In [2]:
import datetime

def convert_timestamp_to_datetime(timestamp):
    timestamp_in_seconds = timestamp / 1000.0
    dt = datetime.datetime.fromtimestamp(timestamp_in_seconds)
    return dt
convert_timestamp_to_datetime(1746624350160)

datetime.datetime(2025, 5, 7, 21, 25, 50, 160000)

In [41]:
def load_jsonl(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in tqdm(f):
            data.append(json.loads(line))
    return data


In [42]:

def mall_jsonl2csv(jsonl_file, csv_file, delimiter=','):
    vacant_list = []
    data = load_jsonl(jsonl_file)
    headers = [     
            'page', 'id', 
            'tag', 
            # 'tag_name',
            'group', 
            'cate1', 'cate1_name',
            'cate2', 'cate2_name',
            'status', 
            'name', 'lat_name', 'sn', 
            'author', 
            'price', 'usd_price', 'profit_rate',
            'amount', 'size', 'condition', 'area', 
            'thumb', 'thumb_big', 
            'desc', 'descEn', 'content', 'created_at', 'updated_at', 'indexShow', 
            'cate_name', 'cate_lat_name',  
            'cls_ph', 'cls_cl1', 'cls_cl2', 'cls_cl3', 
            'cls_or1', 'cls_or2', 'cls_or3', 'cls_or4', 'cls_or5', 
            'cls_fa1', 'cls_ch', 
               ]
    for entry in tqdm(data, desc="Processing JSONL"):
        if len(entry) <= 1:
            vacant_list.append(entry["id"])
            continue
        # page
        entry["page"] = f"https://www.ganvana.com/mall/goods/{entry['id']}"
        # classification
        classification = ["ph", "cl1", "cl2", "cl3", "or1", "or2", "or3", "or4", "or5", "fa1", "ch"]
        if "classification" not in entry or not entry["classification"]:
            entry["classification"] = {}
        for c in classification:
            entry[f"cls_{c}"] = entry["classification"].get(c, '')
        del entry["classification"]
        # cate1, cate2
        cate1 = min(entry.get("cate1", 0), entry.get("cate2", 0))
        cate2 = max(entry.get("cate1", 0), entry.get("cate2", 0))
        entry["cate1"], entry["cate2"] = cate1, cate2
        if cate1 in id_cates:
            entry["cate1_name"] = id_cates[cate1]["name"]
        else:
            entry["cate1_name"] = ''
        if cate2 in id_cates:
            entry["cate2_name"] = id_cates[cate2]["name"]
            entry["cate1_name"] = id_cates[id_cates[cate2]["father"]]["name"]
        else:
            entry["cate2_name"] = ''

        if "name" in entry and entry["name"] and not entry["cate2_name"]:
            for candidates in ["亚纳特螺科","德亚奇螺科", "豆蜗牛科", "小豆螺科", "亚轮螺科", "塔格螺科", "薄泥蜗牛科", "拟阿勇螺科", "吉奥蜗牛科", "管螺科"]:
                if candidates in entry["name"] or candidates[:-1] in entry["name"]:
                    entry["cate2_name"] = name_cates[candidates]["name"]
                    entry["cate1_name"] = id_cates[name_cates[candidates]["father"]]["name"]
                    # print(entry["cate1_name"], entry["cate2_name"], entry["name"])
                    break
        
        if entry.get("price", 0) and entry.get("usd_price", 0):
            entry["profit_rate"] = round((entry["price"] - entry["usd_price"]) / entry["usd_price"], 2)

        #     for name_len in range(len(entry["name"])):
        #         possible_name = entry["name"][name_len:]
        #         if possible_name in name_cates:
        #             entry["cate2_name"] = name_cates[possible_name]["name"]
        #             entry["cate1_name"] = id_cates[name_cates[possible_name]["father"]]["name"]
        #             print(entry["cate1_name"], entry["cate2_name"], entry["name"])
        #             break
        #         elif possible_name+'科' in name_cates:
        #             possible_name = possible_name+'科'
        #             entry["cate2_name"] = name_cates[possible_name]["name"]
        #             entry["cate1_name"] = id_cates[name_cates[possible_name]["father"]]["name"]
        #             print(entry["cate1_name"], entry["cate2_name"], entry["name"])
        #             break
      
        if 'created_at' in entry and isinstance(entry['created_at'], int):
            entry['created_at'] = convert_timestamp_to_datetime(entry['created_at'])
        if 'updated_at' in entry and isinstance(entry['updated_at'], int):
            entry['updated_at'] = convert_timestamp_to_datetime(entry['updated_at'])
    with open(csv_file, 'w', encoding='utf-8') as f:
        firstline = delimiter.join(headers) + '\n'
        f.write(firstline)
        print(firstline)
        for entry in tqdm(data, desc="Converting to CSV"):
            row = delimiter.join([
                    f'"{item}"'.strip() for item in 
                    [str(entry.get(header, '')).strip() for header in headers]
                ])
            f.write(row + '\n')
    with open(jsonl_file.replace('.jsonl', '_vacant.jsonl'), "w", encoding="utf-8") as f:
        json.dump(vacant_list, f, ensure_ascii=False, indent=4)

In [43]:
getGoodsInfo = "D:/_WangKe/scikkk.github.io/projects/ganvana/mall/getGoodsInfo.jsonl"
# mall_jsonl2csv(getGoodsInfo, getGoodsInfo.replace('.jsonl', '.csv'), delimiter=',')

In [44]:

def auction_jsonl2csv(jsonl_file, csv_file, delimiter=','):
    vacant_list = []
    data = load_jsonl(jsonl_file)
    headers = ['page', 'id', 'auction_id', 'status', 
               'family', 'lat_family', 
               'goods_name', 'lat_name', 
               'author', 
               'goods_sn', 'thumb', 'thumb_big', 
               'size', 'product_area', 'desc', 
            #    'more_img', 
               'bid_time', 
               'created_at',
               'winner_who', 'winner_price', 'winner_nickname', 'winner_headimg', 'winner_rec_id'
               ]

    print(headers)
    for entry in tqdm(data, desc="Processing JSONL"):
        if len(entry) <= 1:
            vacant_list.append(entry["id"])
            continue
        
        entry["page"] = f"https://www.ganvana.com/auction/item/{entry['id']}"
        
        if 'created_at' in entry and isinstance(entry['created_at'], int):
            entry['created_at'] = convert_timestamp_to_datetime(entry['created_at'])
            
    with open(csv_file, 'w', encoding='utf-8') as f:
        firstline = delimiter.join(headers) + '\n'
        f.write(firstline)
        print(firstline)
        for entry in tqdm(data, desc="Converting to CSV"):
            row = delimiter.join([
                    f'"{item}"'.strip() for item in 
                    [str(entry.get(header, '')).strip() for header in headers]
                ])
            f.write(row + '\n')
    with open(jsonl_file.replace('.jsonl', '_vacant.jsonl'), "w", encoding="utf-8") as f:
        json.dump(vacant_list, f, ensure_ascii=False, indent=4)

In [45]:
getGoodsInfo = "D:/_WangKe/scikkk.github.io/projects/ganvana/auction/getItem.jsonl"
auction_jsonl2csv(getGoodsInfo, getGoodsInfo.replace('.jsonl', '.csv'), delimiter=',')

41852it [00:00, 55666.76it/s]


['page', 'id', 'auction_id', 'status', 'family', 'lat_family', 'goods_name', 'lat_name', 'author', 'goods_sn', 'thumb', 'thumb_big', 'size', 'product_area', 'desc', 'bid_time', 'created_at', 'winner_who', 'winner_price', 'winner_nickname', 'winner_headimg', 'winner_rec_id']


Processing JSONL: 100%|██████████| 41852/41852 [00:00<00:00, 337517.88it/s]


page,id,auction_id,status,family,lat_family,goods_name,lat_name,author,goods_sn,thumb,thumb_big,size,product_area,desc,bid_time,created_at,winner_who,winner_price,winner_nickname,winner_headimg,winner_rec_id



Converting to CSV: 100%|██████████| 41852/41852 [00:00<00:00, 55786.71it/s]
